In [ ]:
# Setup Inicial DIGITAL HOUSE
try:
    # settings colab:
    import google.colab

    # para colab, cambiar el token de esta url
    ! mkdir -p ../Data
        
except ModuleNotFoundError:    
    # settings local:
    %run "../../../common/0_notebooks_base_setup.py"

Exception: File `'../../../common/0_notebooks_base_setup.py'` not found.

## **GRUPO 6.**


**Pablo Marcelo Ferrero [pabloferrero1991@gmail.com]**

**Bautista Alonso Marangone [bautialonsom@gmail.com]**

**Mauro Montrasi [mauriciomontrasi@gmail.com]**

**Gerardo Maximiliano Burgos [gerarburgos1987@gmail.com]**

Prueba Pablo
---

# **_Desafío 1. Limpieza de datos_**

In [ ]:
# Importamos Librerias de trabajo
import numpy as np
import pandas as pd
import seaborn as sns                       #visualisation
import matplotlib.pyplot as plt             #visualisation
%matplotlib inline                          #visualisation
import re 

In [ ]:
#Importamos la data en un Dataframe
df = pd.read_csv('properati.csv')
#df = pd.read_csv("../Data/properati.csv", ",") 

In [ ]:
df.head(3)

# *1. Análisis Exploratorio*
Revisamos el contenido del Dataframe y sus columnas

In [ ]:
df.shape, df.dtypes

In [ ]:
df.info()

In [ ]:
df.describe()

## *1.1 Nulls*

Consultamos la cantidad de valores nulos:

In [ ]:
df.isnull().sum()

Comparamos faltantes con totales

In [ ]:
cant_nulos_por_campo = df.apply(lambda x: x.isnull().sum(), axis=0)
cant_registros = df.shape[0]
porc_nulos_por_campo = 100 * cant_nulos_por_campo / cant_registros
porc_nulos_por_campo.round(2)


## *1.2 Tipo de propiedad*
Trabajamos sobre la columna_property_type_ para conocer el tipo de unidades presentes en las publicaciones

In [ ]:
# Contamos ocurrencias tipo de propiedad
print("La columna Tipo de Propiedad tiene",df.property_type.value_counts().size,"categorias que son \n",df.property_type.value_counts(dropna=False), "\n y 0 registros Nan")

In [ ]:
#usamos matplot para ver rapidamente la distribucion de unidades
plt.rcParams.update({'font.size': 15})
df["property_type"].value_counts().plot(kind='pie', figsize=(10,10),explode = (0.05, 0, 0, 0), autopct='%1.1f%%',shadow=True, startangle=90)

## *1.3 Tipo de propiedad & Rooms*

In [ ]:
# Rápidamente buscamos outliers valores de rooms por registro 
graf = sns.boxplot(x = df.rooms)

In [ ]:
# Rooms, confirmamos
df.rooms.sort_values(ascending = False).head(4)
dfg6.rooms.describe() 

In [ ]:
#Revisamos
df.shape, dfg6.shape

In [ ]:
#Corroboramos rápidamente
graf2 = sns.boxplot(x = dfg6.rooms)

In [ ]:
dfg6.rooms.describe() 

## *1.4 Distribución de propiedades por zona*

In [ ]:
#Graficamos para tener una vista rapida
df["state_name"].value_counts().plot(kind='bar', figsize=(35,5), )

In [ ]:
#Vemos cantidad de propiedades por zona
df_pxz= df.groupby(['state_name','property_type'])
prop_zona= df_pxz.size().sort_values(ascending=False)
prop_zona

# *2. Limpieza de Datos*

columna Floor

In [ ]:
#Floor tiene datos confusos, mezcla cantidad de pisos con numero de piso dando lugar a interprentaciones erroneas.
print("Floor tiene",df['floor'].notnull().sum(), "notnulls y valores int sample \n", df['floor'][df['floor'].notnull()].sample(8))

columna Country

In [ ]:
# son todos los registros "Argentina"
country_mask = df.country_name == "Argentina"
country_mask.value_counts()

columna Unnamed:0

In [ ]:
#Unnamed: 0 No tiene info relevante, solo repite el index
df["Unnamed: 0"]

columna Operation

In [ ]:
# Son todas operaciones sell
opermask = df.operation !="sell"
opermask.value_counts()

columna Expenses

In [ ]:
# Más del 85% son Nan
df.expenses.isnull().sum()

columna url

In [ ]:
# properati url - no es pertinente para este análisis
df.properati_url.describe()

columna Image

In [ ]:
# Image Thumbail - no es pertinente para este momento análisis
df.image_thumbnail.size, df.image_thumbnail.notnull().sum(), df.image_thumbnail.sample(3)

## *2.1 Drop Columns*

### *dfg6 será nuestra primer isntancia de dataframe a editar*

In [ ]:
#Drop Columns
dfg6 = df.drop(['floor', 'country_name', 'Unnamed: 0', 'operation','expenses', 'properati_url', 'image_thumbnail'], axis=1)
dfg6.shape

In [ ]:
#REVISAMOS CANTIDAD REGISTROS X COL
dfg6.count()

## *2.2 Friendly Columns*

In [ ]:
#REVISO LOS VALORES
list(dfg6.columns.values)
#REORDENO LAS COLUMNAS
dfg6 = dfg6[['title', 'description','property_type','rooms','surface_total_in_m2','surface_covered_in_m2', 'place_name','place_with_parent_names',
'state_name','geonames_id','lat-lon','lat','lon','price','currency','price_aprox_local_currency','price_aprox_usd','price_usd_per_m2', 'price_per_m2']]
#LOOK
dfg6.sample(3)

# *3. Ubicación*
Siguiendo hipotesis revisamos el top ten de localidades con publicaciones

In [ ]:
#top ten de regiones con publicacioens
state_grouped = df.groupby('state_name')
blout = state_grouped["state_name"].size().sort_values(ascending=False)[0:8] 
blout

# *3.1 LOCATION*

Decido dividir la información en dataframes mas pequeños con mayor información relevante. 
'capitalFederal', 'GBA' y uno unificado 'Properati'.

In [ ]:
capitalFederal = df.loc[df.state_name.str.contains("Capital Federal", na=False),:]
GBA =df.loc[df.state_name.str.contains("G.B.A", na=False),:]
properati = pd.concat([capitalFederal,GBA])

Tiro un primer mapeo general, para ver la distibucion de mis datos en un mapa

In [ ]:
import plotly.express as px
import plotly.io as pio
import plotly.offline as pyo

pyo.init_notebook_mode()

In [ ]:
gps_df= properati_xy

In [ ]:
api_token = input("Enter API from Mapbox")

In [ ]:
fig = px.scatter_mapbox(gps_df, lat="lat", lon="lon",
                  color_continuous_scale=["black", "purple", "red" ], size_max=30, zoom=12.5,
                  height = 800, width = 1000, #center = dict(lat = g.center)
                        title='Properati Dataframe',
                       #mapbox_style="open-street-map"
                       )
fig.update_layout(font_size=16,  title={'xanchor': 'center','yanchor': 'top', 'y':0.9, 'x':0.5,}, 
        title_font_size = 24, mapbox_accesstoken=api_token, mapbox_style = "mapbox://styles/strym/ckhd00st61aum19noz9h8y8kw")
fig.update_traces(marker=dict(size=6))

In [ ]:
Ahora, especifico un poco más la información y busco relacion entre mis datos mas refinados

In [ ]:

lat_max = properati.loc[df.lat> -35 ,:]
lat_min = lat_max.loc[lat_max.lat < -34.2]

properati_xy = lat_min.loc[lat_min.lon > -60,:]
size= (25,15)
plt.rcParams["figure.figsize"] = size
properati_xy.plot.hexbin(x='lon', y='lat', C='price_usd_per_m2', reduce_C_function=np.std, gridsize=1000, cmap='winter', fontsize=10)
plt.title('Distribución geografica de valor del m2 en AMBA',fontsize=25)
plt.ylabel('Latitud', fontsize=25)
plt.show();

## *3.2 Place_Name*

*place_name contiene datos confusos, con valores mezclados no siempre pertinetes*

In [ ]:
#reviso los faltantes de columna place_name =23
dfg6.place_name.isnull().sum(), dfg6.place_name.value_counts()

(23,
 Córdoba                                9254
 Rosario                                8504
 Mar del Plata                          6534
 Tigre                                  3324
 Nordelta                               3315
                                        ... 
 Tilisarao                                 1
 Barrio Melazzi                            1
 Tancacha                                  1
 Barrio Cerrado "El Casco de Moreno"       1
 Toledo                                    1
 Name: place_name, Length: 1060, dtype: int64)

## *3.3 Parents_Names*

*Segmentamos la columna place_with_parent_names para trabajar la data de las unidades en virtud de su ubicación, para eso, en primer lugar elegimos las variable relaciondas para tener un panorama de los datos presentes.*


In [ ]:
#Separo el contenido la variable 
location_xtrac = df.place_with_parent_names.str.split("|", expand=True)
location_xtrac.head(4)

,0,1,2,3,4,5,6
0,,Argentina,Capital Federal,Mataderos,,None,None
1,,Argentina,Bs.As. G.B.A. Zona Sur,La Plata,,None,None
2,,Argentina,Capital Federal,Mataderos,,None,None
3,,Argentina,Capital Federal,Liniers,,None,None


## *3.4 New Column BARRIO*

In [ ]:
# Insertamos en ubicación nueva columana Barrio que captura data de place.
dfg6.insert(9,"barrio", location_xtrac[4])


ValueError: cannot insert barrio, already exists

## *3.5 Importo cvs del GCBA con los nombres de los barrios*

Vamos a crear una lista de barrios para extraer los datos de la columana place_name

In [ ]:
#importo la lista de barrios de caba para nuestra segmentacion
barrios = pd.read_csv("../Data/barrios.csv", ",")
maskba = barrios["barrio"]

Rellenamos place_name faltantes con datos de place_with_parent_name para completar ubicacion

# *4. Superficie*

trabajaremos en relación a la dimension, ya que puede tratarse de un destino comercial y no nos interese.¿Interesan > a 1500 m2?

In [ ]:
dfg6.surface_total_in_m2.describe()

In [ ]:
dfsurface = dfg6[["property_type", "surface_total_in_m2", "surface_covered_in_m2", "rooms"]].sort_values(by="surface_total_in_m2", ascending=False)
dfsurface.head(3)

In [ ]:
#quiero ver cuales son mayores
mas_de_mil_metros = dfg6.loc[(dfg6['surface_total_in_m2']>=1500) & (dfg6['surface_covered_in_m2']>=1000),["property_type", "surface_total_in_m2", "surface_covered_in_m2",  "rooms"]]
mas_de_mil_metros.sort_values(by="surface_total_in_m2", ascending=False).head(5)

Me quedo con las menores a 1500 mts y comparo datos para ver su relevancia.

In [ ]:
#me quedo con las menores
dfsurface = dfg6.loc[(dfg6['surface_total_in_m2']<=1500) & (dfg6['surface_covered_in_m2']<=1000),["property_type", "surface_total_in_m2", "surface_covered_in_m2", "rooms"]]
#dfsurface.sort_values( 'surface_total_in_m2', ascending=False).head(5)
dfsurface.surface_total_in_m2.describe()

**Drop Registros mayores a 1500mts**

# *5. Price*

In [ ]:
#comprobamos paridad de variables
dfg6.price_aprox_usd.dtype, dfg6.price.dtype

In [ ]:
#Buscamos Max y Min
hab = pd.cut(dfg6.rooms, [0, 4, 8])
dfg6.pivot_table("price", ['property_type', hab], aggfunc= {min, max} )

In [ ]:
#visualizamos para ver dispersion
sns.boxplot(x=dfg6['price_usd_per_m2'])

In [ ]:
# Visualizamos panorma de la relación zona, tipo, precio superficie
hab1 = pd.cut(dfg6.rooms, [0, 4, 8, 30])
clus = dfg6.pivot_table(['surface_total_in_m2', "price"], ["state_name",'property_type', hab1],aggfunc={'surface_total_in_m2': np.mean,
'price': [min, max]}, fill_value=0)
clus

# *PRECIO*

In [ ]:
properati[['price_aprox_usd']].describe()

Sobre este último trabajo sobre la variable de precios mas fiel. price_usd_per_M2. Separo en quantiles y separo  >.02 y <.95

In [ ]:
properati_filtered = properati[(properati.price_usd_per_m2 < properati.price_usd_per_m2.quantile(.95)) & (properati.price_usd_per_m2 > properati.price_usd_per_m2.quantile(.02))]
properati_filtered.head(1)

In [ ]:
sns.set(style="white", palette="muted", color_codes=True)
rs = np.random.RandomState(10)

# Set up the matplotlib figure
f, axes = plt.subplots(1, 2, figsize=(18, 10))
sns.despine(left=True)


g1=sns.distplot(properati[['price_usd_per_m2']],ax=axes[0], color="red")
g1.axes.set_title('Precios de propiedades extremos', fontsize=24, fontweight=1000)

g2 = sns.distplot(properati_filtered[['price_usd_per_m2']], color="green",ax=axes[1])
g2.axes.set_title('Precios de propiedades grueso', fontsize=24,  color="black", fontweight=1000)

plt.setp(axes, yticks=[])
plt.tight_layout()
plt.show()


Fig.1 muestra precio por m2, sin filtrar arrojando una distribución con muchos casos en 0. A raiz de esto se decidió eliminar los valores nulos. 
Así se puede ver, Fig 2. una distribución, eliminando los cuantiles 0-2 y 95-100, permitiendo visualizar el grueso de los casos

# *6. Combinatoria*

## _6.1 Descripción_
Revisamos la composición de datos de la columna. Utilizamos expresiones para extraer data relevante. 

In [ ]:
dfg6.description.describe()

Abrimos la descripcion para identificar amenities que influyan directamente en el valor del m2

In [ ]:
patron = "pileta" #ampliar expresion
patroncomp = re.compile(patron)
pileta = df.description.apply(lambda x : x if x is np.NaN else patroncomp.search(x))
pileta_notnull= pileta.notnull()
pileta_notnull.sum()


Consultamos si existen registros duplicados, ya que podríamos agrupar por barrios cerrados o complejos

In [ ]:
maskdup1 = df.description.duplicated(keep = False)
dfg6.description[maskdup1].head(6)

## *6.2 Ambientes: Rooms & Tittle*
Trabajamos sobre la variable “rooms”


In [ ]:
capitalFederal_filtro= capitalFederal.property_type.value_counts().size
capitalFederal_filtro=capitalFederal.property_type.value_counts(dropna=False)

In [ ]:
capitalFederal_filtro.plot.barh(color='green', fontsize=15);
plt.title('Cantidad de propiedades en venta en Capital Federal', fontsize=20);
plt.xlabel('Cantidad', fontsize=17);
plt.ylabel('Tipo de propiedad', fontsize=17);
plt.show()

In [ ]:
gba_filtro=GBA.property_type.value_counts().size
gba_filtro=GBA.property_type.value_counts(dropna=False)

In [ ]:
gba_filtro.plot.barh(color='blue', fontsize=15);
plt.title('Cantidad de propiedades en venta en Gran Buenos Aires', fontsize=20);
plt.xlabel('Cantidad', fontsize=17);
plt.ylabel('Tipo de propiedad', fontsize=17);
plt.show()

In [ ]:
properati_room = properati_filtered.loc[properati_filtered.rooms < 8,['rooms','price_usd_per_m2']]
grouped_rooms = properati_room.groupby('rooms').size().plot(kind='pie', figsize=(5,5),cmap='Accent',fontsize=11)
plt.title('Cantidad de cuartos en Capital Federal y GBA', fontsize = 18)
plt.ylabel('')
plt.show();

In [ ]:
properati_room.groupby('rooms').mean()['price_usd_per_m2'].plot(figsize=(14,4),fontsize=11);
plt.title('Precio en USD/m2 en relacion a la cantidad de cuartos que posee la propiedad', fontsize= 20);
plt.xlabel('Cuartos', fontsize = 14);
plt.ylabel('Precio USD/m2', fontsize=14);
plt.show();

# *7. Optimizacion*

Buscamos filas duplicadas

In [ ]:
#No encontramos filas duplicadas
df_duplicates = df.drop_duplicates()
df.shape, df_duplicates.shape

# *8. Guardo mi dataset*

In [ ]:
df.to_csv('g6_limpio_final.csv', index=False)